## Test File for _project_and_slide_moving functionality

In [ ]:
from io import StringIO
import sys
import numpy as np
import matplotlib.pyplot as plt
from planktos import swarm
from planktos.geom import seg_intersect_2D_multilinear_poly as seg_intersect

# Capture print outputs
class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

# Helper to set t_crit
def set_t_crit():
    if t_rot is not None and t_edge is not None:
        return max(t_rot, t_edge)
    elif t_rot is not None:
        return t_rot
    elif t_edge is not None:
        return t_edge
    else:
        return None

# Create a function that will plot the geometry
def plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit=None, newstartpt=None, newendpt=None):
    # unpack intersection
    x = intersection[0]    # (x,y) coordinates of intersection
    t_I = intersection[1]  # btwn 0 & 1, fraction of movement traveled so far
    Q0 = intersection[2]   # edge of mesh element at time of intersection
    Q1 = intersection[3]   # edge of mesh element at time of intersection
    idx = intersection[4]  # index into close_mesh_start/end above
    
    # plot start and end position of mesh
    plt.plot(start_mesh[:,:,0].flatten(), start_mesh[:,:,1].flatten(), 'o-', color='lightgrey', label='startmesh')
    plt.plot(end_mesh[:,:,0].flatten(), end_mesh[:,:,1].flatten(), 'o-', color='black', label='endmesh')
    # plot original start and end position of agent
    plt.plot((startpt[0], endpt[0]), (startpt[1], endpt[1]), 'o-', label='orig trajectory')
    # plot element that was intersected at time of intersection
    plt.plot((Q0[0], Q1[0]), (Q0[1], Q1[1]), label='element at intersection')

    # if a critical time is given, plot the relevant mesh element at that time as well
    if t_crit is not None:
        interp_mesh_el = start_mesh[idx]*(1-t_crit) + end_mesh[idx]*t_crit
        plt.plot((interp_mesh_el[0,0],interp_mesh_el[1,0]),(interp_mesh_el[0,1],interp_mesh_el[1,1]), label='element at t_crit')
    # if new start and end points are given, plot the new trajectory off the element
    if newstartpt is not None:
        plt.plot((newstartpt[0], newendpt[0]), (newstartpt[1], newendpt[1]), 'o-', color='goldenrod', label='new trajectory')
    
    # plot point of intersection
    plt.plot(x[0], x[1], 'x', color='black')
    # plot final point
    plt.plot(finalendpt[0], finalendpt[1], 's', color='black')
    

In [ ]:
# convex mesh elements
start_mesh = np.array([
    [[0.8, 0.2], [0.8, 0.9]],
    [[0.8, 0.9], [0.9, 1.2]]
])
end_mesh = np.array([
    [[0.8, 0.2], [0.8, 0.9]],
    [[0.8, 0.9], [1, 1.2]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.25])
# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
with Capturing() as output:
    finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 1, 1)
t_rot = None; t_edge = None
for assign in output:
    exec(assign)
t_crit = set_t_crit()
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit, newstartpt, newendpt)
print(newstartpt)

In [ ]:
import pdb; pdb.pm()

In [ ]:
# single mesh element, slide off, moving, deforming
start_mesh = np.array([
    [[0.8,0], [0.8,0.8]]
])
end_mesh = np.array([
    [[1.2,0], [1.2,1.1]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.25])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 10, 10)
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt, t_crit=0.6032608695652174, newstartpt=(1.04130335, 0.98097826), newendpt=(1.35869465, 1.21902174))
#plt.plot((1.06666657,1.33333323),(1,1.2))
print(finalendpt)

In [ ]:
# in-line mesh elements, straight-on-hit, moving and non-moving
start_mesh = np.array([
    [[0.5,0], [0.5,1]],
    [[0.5,1], [0.5,2]]
])
end_mesh = np.array([
    [[1.5,0], [1.5,1]],
    [[1.5,1], [1.5,2]]
])
startpt = np.array([0,0.5])
endpt = np.array([2,0.5])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 10, 10)
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt)

In [ ]:
# in-line mesh elements, angled approach, moving and non-moving
start_mesh = np.array([
    [[0.8,0], [0.8,1]],
    [[0.8,1], [0.8,2]]
])
end_mesh = np.array([
    [[1.2,0], [1.2,1]],
    [[1.2,1], [1.2,2]]
])
startpt = np.array([0.5,0.5])
endpt = np.array([1.5,1.25])

# get intersection, project_and_slide
intersection = seg_intersect(startpt, endpt, start_mesh[:,0,:], start_mesh[:,1,:], end_mesh[:,0,:], end_mesh[:,1,:])
finalendpt = swarm._project_and_slide_moving(startpt, endpt, intersection, start_mesh, end_mesh, 10, 10)
plot_it(startpt, endpt, start_mesh, end_mesh, intersection, finalendpt)
print(finalendpt)